In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv', nrows=60000)
data.head(n=2)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.093,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.389,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518


In [3]:
x_train, x_test = train_test_split(
    data.drop(["ID_code", "target"], axis=1), train_size=0.75, shuffle=True, random_state=1,
)

y_train, y_test = train_test_split(
    data["target"], train_size=0.75, shuffle=True, random_state=1,
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 45000 rows, 200 cols
x_test.shape = 15000 rows, 200 cols


In [4]:
pipeline = Pipeline(
    steps=[
        ("scaling", StandardScaler()),
        ("model", LogisticRegression(random_state=27))
    ]
)

pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaling', StandardScaler()),
                ('model', LogisticRegression(random_state=27))])

In [5]:
train_score = roc_auc_score(y_train, pipeline.predict_proba(x_train)[:, 1])
test_score = roc_auc_score(y_test, pipeline.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.864, Test-score: 0.846


## Валидация на отложенной и тестовой выборках

In [6]:
x_train, x_valid = train_test_split(
    data.drop(["ID_code", "target"], axis=1), train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    data["target"], train_size=0.7, shuffle=True, random_state=1,
)

x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=27
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=27
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 42000 rows, 200 cols
x_valid.shape = 12600 rows, 200 cols
x_test.shape = 5400 rows, 200 cols


In [7]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaling', StandardScaler()),
                ('model', LogisticRegression(random_state=27))])

In [8]:
train_score = roc_auc_score(y_train, pipeline.predict_proba(x_train)[:, 1])
valid_score = roc_auc_score(y_valid, pipeline.predict_proba(x_valid)[:, 1])
test_score = roc_auc_score(y_test, pipeline.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.864, Valid-score: 0.853, Test-score: 0.836


## KFold кросс-валидация

In [9]:
kfold = KFold(n_splits=10, shuffle=True, random_state=27)

cv = cross_val_score(
    estimator=pipeline,
    X=data.drop(["ID_code", "target"], axis=1),
    y=data["target"],
    scoring="roc_auc",
    cv=kfold
)

print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.8553 +/- 0.007


In [10]:
print(cv)

[0.85765454 0.86016764 0.84786461 0.85616674 0.85565036 0.86268263
 0.85306287 0.85539241 0.8640899  0.84032007]


## Stratified KFold валидация

In [11]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=27)

cv = cross_val_score(
    estimator=pipeline,
    X=data.drop(["ID_code", "target"], axis=1),
    y=data["target"],
    scoring="roc_auc",
    cv=kfold
)

print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.8553 +/- 0.007


In [12]:
print(cv)

[0.85765454 0.86016764 0.84786461 0.85616674 0.85565036 0.86268263
 0.85306287 0.85539241 0.8640899  0.84032007]


## Комбинация отложенной выборки и кросс-валидации

In [13]:
x_train, x_test = train_test_split(
    data.drop(["ID_code", "target"], axis=1), train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_test = train_test_split(
    data["target"], train_size=0.7, shuffle=True, random_state=1,
)

kfold = KFold(n_splits=10, shuffle=True, random_state=27)

In [14]:
cv = cross_val_score(
    estimator=pipeline,
    X=x_train,
    y=y_train,
    scoring="roc_auc",
    cv=kfold
)

print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.8571 +/- 0.01


In [15]:
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict_proba(x_test)[:, 1]
score = roc_auc_score(y_test, y_pred)

print(f"Out-Of-Fold-score: {round(score, 3)}")

Out-Of-Fold-score: 0.848


In [16]:
delta = np.mean(cv) - score
print(f"Delta between CV-score and OOF-score: {round(delta, 4)}")

Delta between CV-score and OOF-score: 0.0093
